<a href="https://colab.research.google.com/github/harshilgandhi3199/LinearRegression/blob/master/ML_Prac1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt

dataset=pd.read_csv("/content/Bengaluru_House_Data.csv")
dataset.head()

#check if any missing or null values in the dataset
dataset.isnull().sum() 
dataset.shape #(13274,9)

#column "soceity" and has more number of null values and it does not account for any coorelation between variables thus removing it 
#also "availibity" does not account for much change in price thus removing it
#removing "location" becasue it has large number of categories and encoding it will be of no use as it will on add to complexity.
df=dataset.drop(columns=['society', 'availability', 'location'])
df.shape #(13320,6)

#cleaning "size" col i.e 2BHK -> 2
import re
df['size'].fillna("nan", inplace=True)
for i in range(len(df)):
  if df.iloc[i,1]!="nan":
    temp=re.findall(r'\d+', df._get_value(i, 'size'))
    temp=list(map(int, temp))
    temp=temp[0]
    df._set_value(i, 'size', temp)
  else:
    continue

#dropping rows which have Nan
df=df.dropna()
df.head()
df.shape #(12669,6)

X=df.iloc[:, 0:5].values
y=df.iloc[:, -1].values
X

#encoding categorical data i.e string to 0,1,.. and so on
#Label encoding 0th col -> onehot encoding 0th col using ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
labelEncoder=LabelEncoder()
X[:,0]=labelEncoder.fit_transform(X[:,0])
transformer=ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0])], remainder='passthrough')
X=transformer.fit_transform(X.tolist())
#avoding dummy variable trap
X=X[:,1:]

#splitting the dataset into training and train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=0)

#normaliation - appyling min-max normalization
from sklearn.preprocessing import MinMaxScaler
norm=MinMaxScaler().fit(X_train)
X_train_norm=norm.transform(X_train)
X_test_norm=norm.transform(X_test)

#fitting simple linear regression to training set
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train, y_train)

#predicting the test set
y_pred=regressor.predict(X_test)

#score
print("Regressor score: %.2f" % regressor.score(X_test, y_test))

#calculating mean squared error, coefficients and r2score
from sklearn.metrics import mean_squared_error, r2_score
print("Mean_squared_error:  %.2f" % mean_squared_error(y_test,y_pred))
print("Coeefficient of determination:  %.2f" % r2_score(y_test, y_pred))
print("Coefficients: \n", regressor.coef_)

#finding the correlation between features using heatmap
corr_matrix=df.corr()
sns.heatmap(corr_matrix, annot=True)
#we can viualize from the heat map that "total_sqft" and "price" columns are highly correlated and thus "total_sqft" will have
#larger influence on the price of the property

#determining the pearson's correlation between "total_sqft" and "price" 
from scipy.stats import pearsonr
corr,_=pearsonr(df['total_sqft'], df['price'])    #corr (total_sqft<->price = 0.558)
print("Pearson correlation: %.3f" % corr)  #heat map shows the similar results

x1=X_test.astype('float64')[:,4]
y1=y_test.astype('float64')

#visualizing test set results for variable "bath" and "price" because they comparatively more 
#correlated than others
#best fit line using polyfit()
m, b = np.polyfit(x1, y1, 1)
print("Slope: " + str(m) + " Intercept: " + str(b)) 
plt.scatter(X_test[1:15, 4], y_test[1:15], color='red')                       #scatter it along x and y axis
plt.plot(X_test[1:15, 4], m*X_test[1:15, 4] + b, color="blue")
#plt.plot(X_test[1:15, 5], y_pred[1:15], color='blue', linewidth=3)
plt.xlabel("Total_sqft")
plt.ylabel("Price")
plt.show()

#variables that are independent
#correlation between variables
"""
           | total_sqft 	bath 	balcony 	price
total_sqft |	1.000000 	0.394496 	0.145870 	0.558449
bath       |  0.394496 	1.000000 	0.203675 	0.459314
balcony    | 	0.145870 	0.203675 	1.000000 	0.120400
price      |  0.558449 	0.459314 	0.120400 	1.000000
"""

#we can say from above correlation matrix that variables "size" and "area_type" are independent
#and does not consitute for the any correlation. 


#important sources
"""
sklearn website for classes and methods
geeks for geeks - data cleaning and certain other operations
sns website - heatmap
"""

Regressor score: 0.36
